In [1]:
# Client Setup
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
region = os.getenv("AWS_REGION")

client = boto3.client("bedrock-runtime", region_name=region)
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Note: You might have to request access to this model on the AWS Bedrock console
embedding_model_id = "amazon.titan-embed-text-v2:0"

In [6]:
# Chunk by section
import re


def chunk_by_section(document_text):
    pattern = r"\n## "
    return re.split(pattern, document_text)

In [3]:
# Embedding Generation
import json


def generate_embedding(
    text,
    embedding_model_id="amazon.titan-embed-text-v2:0",
    dimensions=1024,
    normalize=True,
):
    request_body = {
        "inputText": text,
        "dimensions": dimensions,
        "normalize": normalize,
    }

    request_json = json.dumps(request_body)
    response = client.invoke_model(
        modelId=embedding_model_id,
        body=request_json,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())

    return response_body["embedding"]

In [20]:
# VectorIndex implementation
import math
from typing import Callable, Optional, Any, List, Dict, Tuple


class VectorIndex:
    def __init__(
        self,
        distance_metric: str = "cosine",
        embedding_fn: Optional[Callable[[str], List[float]]] = None,
    ):
        self.vectors: List[List[float]] = []
        self.documents: List[Dict[str, Any]] = []
        self._vector_dim: Optional[int] = None
        if distance_metric not in ["cosine", "euclidean"]:
            raise ValueError("distance_metric must be 'cosine' or 'euclidean'")
        self._distance_metric = distance_metric
        self._embedding_fn = embedding_fn

    def add_document(self, document: Dict[str, Any]):
        if not self._embedding_fn:
            raise ValueError(
                "Embedding function not provided during initialization."
            )
        if not isinstance(document, dict):
            raise TypeError("Document must be a dictionary.")
        if "content" not in document:
            raise ValueError(
                "Document dictionary must contain a 'content' key."
            )

        content = document["content"]
        if not isinstance(content, str):
            raise TypeError("Document 'content' must be a string.")

        vector = self._embedding_fn(content)
        self.add_vector(vector=vector, document=document)

    def search(
        self, query: Any, k: int = 1
    ) -> List[Tuple[Dict[str, Any], float]]:
        if not self.vectors:
            return []

        if isinstance(query, str):
            if not self._embedding_fn:
                raise ValueError(
                    "Embedding function not provided for string query."
                )
            query_vector = self._embedding_fn(query)
        elif isinstance(query, list) and all(
            isinstance(x, (int, float)) for x in query
        ):
            query_vector = query
        else:
            raise TypeError(
                "Query must be either a string or a list of numbers."
            )

        if self._vector_dim is None:
            return []

        if len(query_vector) != self._vector_dim:
            raise ValueError(
                f"Query vector dimension mismatch. Expected {self._vector_dim}, got {len(query_vector)}"
            )

        if k <= 0:
            raise ValueError("k must be a positive integer.")

        if self._distance_metric == "cosine":
            dist_func = self._cosine_distance
        else:
            dist_func = self._euclidean_distance

        distances = []
        for i, stored_vector in enumerate(self.vectors):
            distance = dist_func(query_vector, stored_vector)
            distances.append((distance, self.documents[i]))

        distances.sort(key=lambda item: item[0])

        return [(doc, dist) for dist, doc in distances[:k]]

    def add_vector(self, vector: List[float], document: Dict[str, Any]):
        if not isinstance(vector, list) or not all(
            isinstance(x, (int, float)) for x in vector
        ):
            raise TypeError("Vector must be a list of numbers.")
        if not isinstance(document, dict):
            raise TypeError("Document must be a dictionary.")
        if "content" not in document:
            raise ValueError(
                "Document dictionary must contain a 'content' key."
            )

        if not self.vectors:
            self._vector_dim = len(vector)
        elif len(vector) != self._vector_dim:
            raise ValueError(
                f"Inconsistent vector dimension. Expected {self._vector_dim}, got {len(vector)}"
            )

        self.vectors.append(list(vector))
        self.documents.append(document)

    def _euclidean_distance(
        self, vec1: List[float], vec2: List[float]
    ) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")
        return math.sqrt(sum((p - q) ** 2 for p, q in zip(vec1, vec2)))

    def _dot_product(self, vec1: List[float], vec2: List[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")
        return sum(p * q for p, q in zip(vec1, vec2))

    def _magnitude(self, vec: List[float]) -> float:
        return math.sqrt(sum(x * x for x in vec))

    def _cosine_distance(self, vec1: List[float], vec2: List[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")

        mag1 = self._magnitude(vec1)
        mag2 = self._magnitude(vec2)

        if mag1 == 0 and mag2 == 0:
            return 0.0
        elif mag1 == 0 or mag2 == 0:
            return 1.0

        dot_prod = self._dot_product(vec1, vec2)
        cosine_similarity = dot_prod / (mag1 * mag2)
        cosine_similarity = max(-1.0, min(1.0, cosine_similarity))

        return 1.0 - cosine_similarity

    def __len__(self) -> int:
        return len(self.vectors)

    def __repr__(self) -> str:
        has_embed_fn = "Yes" if self._embedding_fn else "No"
        return f"VectorIndex(count={len(self)}, dim={self._vector_dim}, metric='{self._distance_metric}', has_embedding_fn='{has_embed_fn}')"

In [5]:
with open("./003_report.md", "r") as f:
    text = f.read()

In [7]:
# 1. Chunk the text by section
chunks = chunk_by_section(text)

In [8]:
len(chunks)

15

In [10]:
chunks[3]

'Methodology\n\nThe insights compiled within this Annual Interdisciplinary Research Review represent a synthesis of findings drawn from standard departmental reporting cycles, specialized project updates, and cross-functional review meetings conducted throughout the year. Data sources included internal project databases, laboratory notebooks, financial reporting systems, legal case summaries, security incident logs, and minutes from dedicated working groups. A central review committee, comprising representatives nominated by each division head, was tasked with identifying key developments and potential cross-domain implications. This committee utilized a standardized reporting template to capture essential details, including unique identifiers (project codes, error numbers, case references, etc.) and progress metrics. Subsequent analysis focused on identifying thematic overlaps, shared challenges, and opportunities for synergistic development, forming the basis of this consolidated rep

In [11]:
# 2. Generate embeddings for each chunk

embeddings = [generate_embedding(chunk) for chunk in chunks]

In [12]:
len(embeddings)

15

In [13]:
len(embeddings[0])

1024

In [14]:
embeddings[0][0:10]

[-0.058282431215047836,
 0.011692947708070278,
 0.03184407576918602,
 0.006079585291445255,
 -0.020468125119805336,
 -0.011550065129995346,
 -0.012378472834825516,
 0.011308414861559868,
 -0.07636644691228867,
 0.016307353973388672]

In [15]:
# 3. Create a vector store and add each embedding (and the chunk content) to it
store = VectorIndex()

for embedding, chunk in zip(embeddings, chunks):
    store.add_vector(embedding, {"content": chunk})

We store the embeddings and the original text. This way when we want to find the similarity, it will use the embeddings & when we want to know which text is similar it can pull up the relevant text.

In [16]:
# 4. Some time later, a user will ask a question. Generate an embedding for it
user_embedding = generate_embedding("What did the software engineering dept do last year?")

In [19]:
# 5. Search the store with the embedding, find the 2 most relevant chunks
results = store.search(user_embedding, 2)

for doc, distance in results:
    print("Cosine distance: ", distance, "\n", doc["content"][0:200], "\n-----\n")

Cosine distance:  0.6907249165167906 
 Section 2: Software Engineering - Project Phoenix Stability Enhancements

The Software Engineering division dedicated considerable effort to improving the stability and performance of the core systems 
-----

Cosine distance:  0.7210594524998164 
 Methodology

The insights compiled within this Annual Interdisciplinary Research Review represent a synthesis of findings drawn from standard departmental reporting cycles, specialized project updates 
-----



The search results above returned embeddings with the lowest cosine distance (highest cosine similarity).